<a href="https://colab.research.google.com/github/cedro3/others2/blob/main/Cartoon_StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.セットアップ

In [ ]:
#@title githubコピー ＆ ライブラリー・インストール
# githubからコードをコピー
!git clone https://github.com/happy-jihye/Cartoon-StyleGan2.git
%cd Cartoon-StyleGan2

# ライブラリーのインストール
!pip install ftfy regex tqdm ninja gdown
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
#@title 学習済みパラメータのダウンロード
import os
import gdown
os.makedirs('networks', exist_ok=True)
gdown.download('https://drive.google.com/uc?id=1PQutd-JboOCOZqmd95XWxWrO8gGEvRcO', './networks/ffhq256.pt', quiet=False)
gdown.download('https://drive.google.com/uc?id=1QQuZGtHgD24Dn5E21Z2Ik25EPng58MoU', './networks/encoder_ffhq.pt', quiet=False)

gdown.download('https://drive.google.com/uc?id=1Sa4ahVKnzs1DKnNRo0vs4fZBcrivqMoY', './pt.zip', quiet=False)
! unzip pt.zip -d networks

# 2.補完動画作成

In [ ]:
#@title 画像生成（swap無し）
import os
import torch
import cv2 
import numpy as np
from torchvision.utils import save_image, make_grid
import PIL.Image as pilimg

from model import Generator


device='cuda'
n_sample=5
truncation = 0.7


# =============================================

# Genearaotr1
network1 = 'ffhq256'  #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network1 = f'/content/Cartoon-StyleGan2/networks/{network1}.pt' 
network1 = torch.load(network1)

g1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g1.load_state_dict(network1["g_ema"], strict=False)
trunc1 = g1.mean_latent(4096)

# Generator2
network2 = 'NaverWebtoon_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2 = f'/content/Cartoon-StyleGan2/networks/{network2}.pt' 
network2 = torch.load(network2)

g2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g2.load_state_dict(network2["g_ema"], strict=False)
trunc2 = g2.mean_latent(4096)

# Generator3
network3 = 'Disney_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network3 = f'/content/Cartoon-StyleGan2/networks/{network3}.pt' 
network3 = torch.load(network3)

g3 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g3.load_state_dict(network3["g_ema"], strict=False)
trunc3 = g3.mean_latent(4096)

# Generator4
network4 = 'Metface_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network4 = f'/content/Cartoon-StyleGan2/networks/{network4}.pt' 
network4 = torch.load(network4)

g4 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g4.load_state_dict(network4["g_ema"], strict=False)
trunc4 = g4.mean_latent(4096)


# =============================================

# directory to save image
outdir = 'results1' #param {type:"string"}
if not os.path.isdir(f'{outdir}'):
   os.makedirs(f'./asset/{outdir}', exist_ok=True)

imgs = []
number_of_img = 11 #@param {type:"slider", min:0, max:30, step:1}
number_of_step = 7 #@param {type:"slider", min:0, max:10, step:1}

swap = False #@param {type:"boolean"}
swap_layer_num = 2 #@param {type:"slider", min:1, max:6, step:1}

with torch.no_grad():

  latent1 = torch.randn(1, 14, 512, device=device)
  latent1 = g1.get_latent(latent1)
  latent_interp = torch.zeros(1, latent1.shape[1], latent1.shape[2]).to(device)

  for i in range(number_of_img):
      # latent1

      latent2 = torch.randn(1, 14, 512, device=device)
      latent2 = g1.get_latent(latent2)


      for j in range(number_of_step):

          latent_interp = latent1 + (latent2-latent1) * float(j/(number_of_step-1))

          imgs_gen1, save_swap_layer = g1([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc1,
                                  swap=swap, swap_layer_num=swap_layer_num,
                                  randomize_noise=False)
          imgs_gen2, _ = g2([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc2,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen3, _ = g3([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc3,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen4, _ = g4([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc4,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )

                          
          grid = make_grid(torch.cat([imgs_gen1, imgs_gen2, imgs_gen3, imgs_gen4], 0),
                            nrow=4,
                            normalize=True,
                            range=(-1,1),
                            )
          
          ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
          im = pilimg.fromarray(ndarr)
          im.save(f'/content/Cartoon-StyleGan2/asset/{outdir}/out-{i*number_of_step+j}.png')


      latent1 = latent2

In [ ]:
#@title  画像をmp4に変換（swap無し）
!ffmpeg -r 11 -i asset/results1/out-%d.png -vcodec libx264 -pix_fmt yuv420p output1.mp4

In [ ]:
#@title mp4動画の再生（swap無し）
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output1.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="90%" height="90%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title 画像生成（swap有り1-2）
import os
import torch
import cv2 
import numpy as np
from torchvision.utils import save_image, make_grid
import PIL.Image as pilimg

from model import Generator


device='cuda'
n_sample=5
truncation = 0.7


# =============================================

# Genearaotr1
network1 = 'ffhq256'  #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network1 = f'/content/Cartoon-StyleGan2/networks/{network1}.pt' 
network1 = torch.load(network1)

g1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g1.load_state_dict(network1["g_ema"], strict=False)
trunc1 = g1.mean_latent(4096)

# Generator2
network2 = 'NaverWebtoon_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2 = f'/content/Cartoon-StyleGan2/networks/{network2}.pt' 
network2 = torch.load(network2)

g2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g2.load_state_dict(network2["g_ema"], strict=False)
trunc2 = g2.mean_latent(4096)

# Generator3
network3 = 'Disney_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network3 = f'/content/Cartoon-StyleGan2/networks/{network3}.pt' 
network3 = torch.load(network3)

g3 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g3.load_state_dict(network3["g_ema"], strict=False)
trunc3 = g3.mean_latent(4096)

# Generator4
network4 = 'Metface_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network4 = f'/content/Cartoon-StyleGan2/networks/{network4}.pt' 
network4 = torch.load(network4)

g4 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g4.load_state_dict(network4["g_ema"], strict=False)
trunc4 = g4.mean_latent(4096)


# =============================================

# directory to save image
outdir = 'results2' #param {type:"string"}
if not os.path.isdir(f'{outdir}'):
   os.makedirs(f'./asset/{outdir}', exist_ok=True)

imgs = []
number_of_img = 11 #@param {type:"slider", min:0, max:30, step:1}
number_of_step = 7 #@param {type:"slider", min:0, max:10, step:1}

swap = True #@param {type:"boolean"}
swap_layer_num = 2 #@param {type:"slider", min:1, max:6, step:1}

with torch.no_grad():

  latent1 = torch.randn(1, 14, 512, device=device)
  latent1 = g1.get_latent(latent1)
  latent_interp = torch.zeros(1, latent1.shape[1], latent1.shape[2]).to(device)

  for i in range(number_of_img):
      # latent1

      latent2 = torch.randn(1, 14, 512, device=device)
      latent2 = g1.get_latent(latent2)


      for j in range(number_of_step):

          latent_interp = latent1 + (latent2-latent1) * float(j/(number_of_step-1))

          imgs_gen1, save_swap_layer = g1([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc1,
                                  swap=swap, swap_layer_num=swap_layer_num,
                                  randomize_noise=False)
          imgs_gen2, _ = g2([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc2,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen3, _ = g3([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc3,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen4, _ = g4([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc4,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )

                          
          grid = make_grid(torch.cat([imgs_gen1, imgs_gen2, imgs_gen3, imgs_gen4], 0),
                            nrow=4,
                            normalize=True,
                            range=(-1,1),
                            )
          
          ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
          im = pilimg.fromarray(ndarr)
          im.save(f'/content/Cartoon-StyleGan2/asset/{outdir}/out-{i*number_of_step+j}.png')


      latent1 = latent2

In [ ]:
#@title  画像をmp4に変換（swap有り1-2）
!ffmpeg -r 11 -i asset/results2/out-%d.png -vcodec libx264 -pix_fmt yuv420p output2.mp4

In [ ]:
#@title mp4動画の再生（swap有り1-2）
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output2.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="90%" height="90%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [ ]:
#@title 画像生成（swap有り1-4）
import os
import torch
import cv2 
import numpy as np
from torchvision.utils import save_image, make_grid
import PIL.Image as pilimg

from model import Generator


device='cuda'
n_sample=5
truncation = 0.7


# =============================================

# Genearaotr1
network1 = 'ffhq256'  #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network1 = f'/content/Cartoon-StyleGan2/networks/{network1}.pt' 
network1 = torch.load(network1)

g1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g1.load_state_dict(network1["g_ema"], strict=False)
trunc1 = g1.mean_latent(4096)

# Generator2
network2 = 'NaverWebtoon_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2 = f'/content/Cartoon-StyleGan2/networks/{network2}.pt' 
network2 = torch.load(network2)

g2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g2.load_state_dict(network2["g_ema"], strict=False)
trunc2 = g2.mean_latent(4096)

# Generator3
network3 = 'Disney_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network3 = f'/content/Cartoon-StyleGan2/networks/{network3}.pt' 
network3 = torch.load(network3)

g3 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g3.load_state_dict(network3["g_ema"], strict=False)
trunc3 = g3.mean_latent(4096)

# Generator4
network4 = 'Metface_StructureLoss' #param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network4 = f'/content/Cartoon-StyleGan2/networks/{network4}.pt' 
network4 = torch.load(network4)

g4 = Generator(256, 512, 8, channel_multiplier=2).to(device)
g4.load_state_dict(network4["g_ema"], strict=False)
trunc4 = g4.mean_latent(4096)


# =============================================

# directory to save image
outdir = 'results3' #param {type:"string"}
if not os.path.isdir(f'{outdir}'):
   os.makedirs(f'./asset/{outdir}', exist_ok=True)

imgs = []
number_of_img = 11 #@param {type:"slider", min:0, max:30, step:1}
number_of_step = 7 #@param {type:"slider", min:0, max:10, step:1}

swap = True #@param {type:"boolean"}
swap_layer_num = 4 #@param {type:"slider", min:1, max:6, step:1}

with torch.no_grad():

  latent1 = torch.randn(1, 14, 512, device=device)
  latent1 = g1.get_latent(latent1)
  latent_interp = torch.zeros(1, latent1.shape[1], latent1.shape[2]).to(device)

  for i in range(number_of_img):
      # latent1

      latent2 = torch.randn(1, 14, 512, device=device)
      latent2 = g1.get_latent(latent2)


      for j in range(number_of_step):

          latent_interp = latent1 + (latent2-latent1) * float(j/(number_of_step-1))

          imgs_gen1, save_swap_layer = g1([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc1,
                                  swap=swap, swap_layer_num=swap_layer_num,
                                  randomize_noise=False)
          imgs_gen2, _ = g2([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc2,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen3, _ = g3([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc3,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )
          imgs_gen4, _ = g4([latent_interp],
                                  input_is_latent=True,                                     
                                  truncation=0.7,
                                  truncation_latent=trunc4,
                                  swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                  )

                          
          grid = make_grid(torch.cat([imgs_gen1, imgs_gen2, imgs_gen3, imgs_gen4], 0),
                            nrow=4,
                            normalize=True,
                            range=(-1,1),
                            )
          
          ndarr = grid.mul(255).add_(0.5).clamp_(0, 255).permute(1, 2, 0).to('cpu', torch.uint8).numpy()
          im = pilimg.fromarray(ndarr)
          im.save(f'/content/Cartoon-StyleGan2/asset/{outdir}/out-{i*number_of_step+j}.png')


      latent1 = latent2

In [ ]:
#@title  画像をmp4に変換（swap有り1-4）
!ffmpeg -r 11 -i asset/results3/out-%d.png -vcodec libx264 -pix_fmt yuv420p output3.mp4

In [ ]:
#@title mp4動画の再生（swap有り1-4）
from IPython.display import HTML
from base64 import b64encode

mp4 = open('./output3.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="90%" height="90%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

# 3.文による顔の変化


In [ ]:
#@title 設定
seed = 356289 #@param {type:"slider", min:0, max:1000000, step:1}

description = 'a really angry face' #@param {type:"string"}

latent_path = None #@param {type:"string"}

optimization_steps = 300 #@param {type:"number"}

l2_lambda = 0.004 #@param {type:"number"}

create_video = False #@param {type:"boolean"}

result_dir = "asset/results2" #@param {type:"string"}

device = 'cuda'

# -----------------------------
args = {
    "seed" : seed,
    "description": description,
    "ckpt": "/content/Cartoon-StyleGan2/networks/ffhq256.pt",
    "ckpt2": "/content/Cartoon-StyleGan2/networks/NaverWebtoon.pt",
    "stylegan_size": 256,
    "latent_dim" : 14,
    "lr_rampup": 0.05,
    "lr": 0.1,
    "step": optimization_steps,
    "l2_lambda": l2_lambda,
    "latent_path": latent_path,
    "truncation": 0.7,
    "save_intermediate_image_every": 1 if create_video else 20,
    "device" : "cuda",
    "results_dir": result_dir,
}

In [ ]:
#@title 最適化実行
from run_optimization import main
from argparse import Namespace

final_result, latent_init2, latent_fin2 = main(Namespace(**args))

In [ ]:
#@title 顔の変化表示（swap無し）
import torch

from model import Generator
from utils import imshow, tensor2image

device = 'cuda'

# ---------------
# Generator
# ---------------

# Generator1
network1='/content/Cartoon-StyleGan2/networks/ffhq256.pt' 
network1 = torch.load(network1)

generator1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator1.load_state_dict(network1["g_ema"], strict=False)

trunc1 = generator1.mean_latent(4096)

# Generator2
Target_network = "NaverWebtoon_StructureLoss" #@param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2=f'/content/Cartoon-StyleGan2/networks/{Target_network}.pt' 
network2 = torch.load(network2)

generator2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator2.load_state_dict(network2["g_ema"], strict=False)

trunc2 = generator2.mean_latent(4096)

# ---------------
# Interpolation
# ---------------

number_of_step = 6 #@param {type:"slider", min:0, max:10, step:1}
latent_interp = torch.zeros(number_of_step, latent_init2.shape[1], latent_init2.shape[2]).to(device)

strength = 2.5 #@param {type:"slider", min:1, max:3, step:0.25}
swap = False #@param {type:"boolean"}
swap_layer_num = 2 #@param {type:"slider", min:1, max:6, step:1}


with torch.no_grad():
    for j in range(number_of_step):

        latent_interp[j] = latent_init2 + strength * (latent_fin2-latent_init2) * float(j/(number_of_step-1))

        imgs_gen1, save_swap_layer = generator1([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                truncation_latent=trunc1,
                                swap=swap, swap_layer_num=swap_layer_num,
                                randomize_noise=False)
        imgs_gen2, _ = generator2([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                truncation_latent=trunc2)
        
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen1], dim=2)), 20)
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen2], dim=2)), 20)

In [ ]:
#@title 顔の変化表示（swap有り1-2）
import torch

from model import Generator
from utils import imshow, tensor2image

device = 'cuda'

# ---------------
# Generator
# ---------------

# Generator1
network1='/content/Cartoon-StyleGan2/networks/ffhq256.pt' 
network1 = torch.load(network1)

generator1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator1.load_state_dict(network1["g_ema"], strict=False)

trunc1 = generator1.mean_latent(4096)

# Generator2
Target_network = "NaverWebtoon_StructureLoss" #@param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2=f'/content/Cartoon-StyleGan2/networks/{Target_network}.pt' 
network2 = torch.load(network2)

generator2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator2.load_state_dict(network2["g_ema"], strict=False)

trunc2 = generator2.mean_latent(4096)

# ---------------
# Interpolation
# ---------------

number_of_step = 6 #@param {type:"slider", min:0, max:10, step:1}
latent_interp = torch.zeros(number_of_step, latent_init2.shape[1], latent_init2.shape[2]).to(device)

strength = 2.5 #@param {type:"slider", min:1, max:3, step:0.25}
swap = True #@param {type:"boolean"}
swap_layer_num = 2 #@param {type:"slider", min:1, max:6, step:1}


with torch.no_grad():
    for j in range(number_of_step):

        latent_interp[j] = latent_init2 + strength * (latent_fin2-latent_init2) * float(j/(number_of_step-1))

        imgs_gen1, save_swap_layer = generator1([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                truncation_latent=trunc1,
                                swap=swap, swap_layer_num=swap_layer_num,
                                randomize_noise=False)
        imgs_gen2, _ = generator2([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                truncation_latent=trunc2)
        
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen1], dim=2)), 20)
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen2], dim=2)), 20)

In [ ]:
#@title 顔の変化表示（swap有り1-4）
import torch

from model import Generator
from utils import imshow, tensor2image

device = 'cuda'

# ---------------
# Generator
# ---------------

# Generator1
network1='/content/Cartoon-StyleGan2/networks/ffhq256.pt' 
network1 = torch.load(network1)

generator1 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator1.load_state_dict(network1["g_ema"], strict=False)

trunc1 = generator1.mean_latent(4096)

# Generator2
Target_network = "NaverWebtoon_StructureLoss" #@param ['ffhq256', 'NaverWebtoon', 'NaverWebtoon_StructureLoss', 'NaverWebtoon_FreezeSG', 'Romance101', 'TrueBeauty', 'Disney', 'Disney_StructureLoss', 'Disney_FreezeSG', 'Metface_StructureLoss', 'Metface_FreezeSG']
network2=f'/content/Cartoon-StyleGan2/networks/{Target_network}.pt' 
network2 = torch.load(network2)

generator2 = Generator(256, 512, 8, channel_multiplier=2).to(device)
generator2.load_state_dict(network2["g_ema"], strict=False)

trunc2 = generator2.mean_latent(4096)

# ---------------
# Interpolation
# ---------------

number_of_step = 6 #@param {type:"slider", min:0, max:10, step:1}
latent_interp = torch.zeros(number_of_step, latent_init2.shape[1], latent_init2.shape[2]).to(device)

strength = 2.5 #@param {type:"slider", min:1, max:3, step:0.25}
swap = True #@param {type:"boolean"}
swap_layer_num = 4 #@param {type:"slider", min:1, max:6, step:1}


with torch.no_grad():
    for j in range(number_of_step):

        latent_interp[j] = latent_init2 + strength * (latent_fin2-latent_init2) * float(j/(number_of_step-1))

        imgs_gen1, save_swap_layer = generator1([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                truncation_latent=trunc1,
                                swap=swap, swap_layer_num=swap_layer_num,
                                randomize_noise=False)
        imgs_gen2, _ = generator2([latent_interp],
                                input_is_latent=True,                                     
                                truncation=0.7,
                                swap=swap, swap_layer_num=swap_layer_num, swap_layer_tensor=save_swap_layer,
                                truncation_latent=trunc2)
        
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen1], dim=2)), 20)
imshow(tensor2image(torch.cat([img_gen for img_gen in imgs_gen2], dim=2)), 20)